# **Ridge - Lasso - ElasticNet**

## Imports & Data

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_boston

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
data = load_boston()

df = pd.DataFrame(data=data.data, columns=data.feature_names)

df["PRICE"] = data.target

c:\Users\mbatu\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the

___
___

## Polinom Dönüştürme

In [71]:
from sklearn.preprocessing import PolynomialFeatures

In [72]:
poly_conv = PolynomialFeatures(degree=2, include_bias=False)

In [73]:
poly_features = poly_conv.fit_transform(X)

___
___

## Train Test Split

In [74]:
X = df.drop("PRICE", axis=1)
y= df["PRICE"]

In [75]:
from sklearn.model_selection import train_test_split

In [76]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.25, random_state=9)

___
___

## Veriyi Ölçeklendirme
(Scaling Data)

In [77]:
from sklearn.preprocessing import StandardScaler

In [78]:
scaler = StandardScaler()

In [79]:
scaler.fit(X_train)

StandardScaler()

In [80]:
X_train = scaler.transform(X_train)

In [81]:
X_test = scaler.transform(X_test)

___
___
## Ridge Regression

In [82]:
from sklearn.linear_model import Ridge

In [83]:
ridge_model = Ridge(alpha=10)

In [84]:
ridge_model.fit(X_train, y_train)

Ridge(alpha=10)

In [85]:
ridge_pred = ridge_model.predict(X_test)

In [86]:
MAE = mean_absolute_error(y_test,ridge_pred)
MSE = mean_squared_error(y_test,ridge_pred)
RMSE = np.sqrt(MSE)

print("MAE  : ", MAE)
print("RMSE : ", RMSE)

MAE  :  2.5089591837981793
RMSE :  3.2521169446067932


In [87]:
ridge_train = ridge_model.predict(X_train)
MAE = mean_absolute_error(y_train,ridge_train)
MAE

2.302763230443299

### Alpha Değerinin Seçimi

In [88]:
from sklearn.linear_model import RidgeCV

In [89]:
ridge_cv = RidgeCV(alphas=(0.01,0.1, 1.0, 10.0), scoring="neg_mean_absolute_error")

In [90]:
ridge_cv.fit(X_train,y_train)

RidgeCV(alphas=(0.01, 0.1, 1.0, 10.0), scoring='neg_mean_absolute_error')

In [91]:
ridge_cv.alpha_

0.1

In [92]:
ridge_cv_pred = ridge_cv.predict(X_test)

In [93]:
MAE = mean_absolute_error(y_test,ridge_cv_pred)
MSE = mean_squared_error(y_test,ridge_cv_pred)
RMSE = np.sqrt(MSE)

print("MAE  : ", MAE)
print("RMSE : ", RMSE)

MAE  :  2.7179335049903313
RMSE :  3.685276114433708


In [94]:
ridge_cv_train = ridge_cv.predict(X_train)
MAE = mean_absolute_error(y_train,ridge_cv_train)
MAE

1.7684937663987919

---
---
## Lasso Regression

In [95]:
from sklearn.linear_model import LassoCV

In [96]:
lasso_cv_model = LassoCV(eps=0.1, n_alphas=100, cv=5)

In [97]:
lasso_cv_model.fit(X_train, y_train)

LassoCV(cv=5, eps=0.1)

In [98]:
lasso_cv_model.alpha_

0.6828621619826489

In [99]:
lasso_cv_model.coef_

array([-0.        ,  0.        , -0.        ,  0.        , -0.        ,
        0.        , -0.        , -0.        , -0.        , -0.        ,
       -1.04569675,  0.        , -0.        , -0.        ,  0.        ,
       -0.        ,  0.33106637, -0.        , -0.33416547, -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -0.        , -0.        ,  0.        ,
       -0.        , -0.        , -0.        , -0.06580273, -0.        ,
       -0.        , -0.        ,  0.        , -0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        ,  0.        , -0.        ,  3.49092082, -0.  

In [100]:
lasso_cv_pred = lasso_cv_model.predict(X_test)

In [101]:
MAE = mean_absolute_error(y_test,lasso_cv_pred)
MSE = mean_squared_error(y_test,lasso_cv_pred)
RMSE = np.sqrt(MSE)

print("MAE  : ", MAE)
print("RMSE : ", RMSE)

MAE  :  3.2682953671193316
RMSE :  4.668293273898157


In [112]:
lasso_cv_train = lasso_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,lasso_cv_train)
MAE

3.189401705531224

---
---
## Elastic Net

In [103]:
from sklearn.linear_model import ElasticNetCV

In [104]:
elasticnet_model = ElasticNetCV(l1_ratio=[0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99, 1], tol=0.01)

In [105]:
elasticnet_model.fit(X_train, y_train)

c:\Users\mbatu\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 297.5764819502731, tolerance: 272.02624026402646
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\mbatu\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 288.045768709012, tolerance: 272.02624026402646
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\mbatu\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 299.8658068966283, tolerance: 235.00122244224417
  model = cd_fast.enet_coordinate_descent_gram(
c:

ElasticNetCV(l1_ratio=[0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99, 1], tol=0.01)

In [106]:
elasticnet_model.l1_ratio_

0.9

In [107]:
elasticnet_model.coef_

array([ 2.19070569e+00, -5.95956868e+00,  3.78137424e+00,  1.86279121e+00,
        1.86101372e+00,  3.69597408e+00,  1.57451886e+00, -8.79212823e-02,
        5.06404175e-02,  0.00000000e+00,  0.00000000e+00,  5.55612670e+00,
       -2.06627814e+00,  1.04568999e+00,  3.41174452e-01,  0.00000000e+00,
        2.57660615e+00, -2.70355387e+00, -0.00000000e+00, -0.00000000e+00,
       -1.34227460e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -8.10523248e-01,  0.00000000e+00,  1.33729471e+00, -0.00000000e+00,
       -3.10713142e-01,  0.00000000e+00,  1.14907275e+00, -1.83303869e-02,
       -8.32740974e-01, -1.27376368e-03,  1.93912312e+00,  2.71495190e+00,
        0.00000000e+00, -6.98218352e-01,  4.12604078e-01, -0.00000000e+00,
        1.11097022e+00, -6.92223592e-01,  3.94393583e-01, -9.42774097e-01,
       -5.70316341e-01,  0.00000000e+00, -2.19451413e+00,  5.21944011e-01,
       -3.01461654e+00,  5.94071392e-01, -3.61822166e+00, -4.14702939e+00,
        2.29463480e-01,  

In [108]:
elastic_pred = elasticnet_model.predict(X_test)

In [109]:
MAE = mean_absolute_error(y_test,elastic_pred)
MSE = mean_squared_error(y_test,elastic_pred)
RMSE = np.sqrt(MSE)

print("MAE  : ", MAE)
print("RMSE : ", RMSE)

MAE  :  2.4684630122956768
RMSE :  3.3977658046265704


In [113]:
elastic_train = elasticnet_model.predict(X_train)
MAE = mean_absolute_error(y_train,elastic_train)
MAE

2.011566112827841

---
---

# **https://www.youtube.com/watch?v=v2Q3Yi6Xca4**